In [1]:
from gensim.models.wrappers import FastText
from gensim.utils import tokenize

In [2]:
fasttext_model = FastText.load('/mnt/A20CC3B20CC37FB1/cc.vi.300.gs')

In [3]:
import re
import unicodedata
import torch
import torch.nn as nn
import torch
from torch.autograd import Variable
from torch import optim
from collections import Counter
import torch.nn.functional as F
import pickle
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
torch.manual_seed(10)
# Enable inline plotting
nltk.download('punkt')
%matplotlib inline

[nltk_data] Downloading package punkt to /home/computer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# if gpu is to be used
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor
Variable = torch.autograd.Variable

In [5]:
file = open('./data/Training_shuf.txt', 'r')
data = []
X = [] # text 
y = [] # label (text)
for line in file:
    row = line.split(' ', 1)
    data.append((row[1].strip(), row[0].split('__')[1]))
    X.append(row[1].strip())
    y.append(row[0].split('__')[1])

In [6]:
le = preprocessing.LabelEncoder()
le.fit(y)
Y_train = le.transform(y)
number_of_class = le.classes_.size
np.save('./data/classes.npy',le.classes_)

In [7]:
def from_pretrained(embeddings, freeze=True):
    assert embeddings.dim() == 2, \
         'Embeddings parameter is expected to be 2-dimensional'
    rows, cols = embeddings.shape
    embedding = torch.nn.Embedding(num_embeddings=rows, embedding_dim=cols)
    embedding.weight = torch.nn.Parameter(embeddings)
    embedding.weight.requires_grad = not freeze
    return embedding
class Model(torch.nn.Module) :
    def __init__(self,weights,hidden_dim,number_of_class) :
        super(Model,self).__init__()
        self.hidden_dim = hidden_dim
        embedding_dim = weights.shape[1]
        self.embeddings = from_pretrained(weights)
        self.lstm1 = nn.GRU(embedding_dim,hidden_dim,dropout=0.2)
        self.linear1 = nn.Linear(hidden_dim,number_of_class)
    def forward(self,inputs,hidden) :
        x = self.embeddings(inputs).view(len(inputs),1,-1)
        lstm_out1,lstm_h1 = self.lstm1(x,hidden)
        x = lstm_out1[-1]
#         x = lstm_out1[:,-1,:].squeeze()
        x = self.linear1(x)
        x = F.log_softmax(x)
        return x,lstm_h1
    def init_hidden(self) :
        return Variable(torch.randn(1, 1, self.hidden_dim),requires_grad=True).type(FloatTensor)
        # return (Variable(torch.zeros(1, 1, self.hidden_dim).type(FloatTensor)),Variable(torch.zeros(1, 1, self.hidden_dim).type(FloatTensor)))
def get_indexes(text):
    return [fasttext_model.wv.vocab.get(t).index for t in tokenize(text) if t in fasttext_model.wv.vocab]
def save_params(model, i):
    pl = list(model.parameters())
    pl = [p for i,p in enumerate(pl) if i > 0]
    torch.save(pl,'./data/params-%d.dat' % i)
def load_params(model, params_path):
    pl = list(model.parameters())
    pl = [p for i,p in enumerate(pl) if i > 0]
    pll = torch.load(params_path)
    for p1, p2 in zip(pl, pll):
        p1.data = p2.data

In [8]:
weights = FloatTensor(fasttext_model.wv.syn0)

In [9]:
model = Model(weights,80,number_of_class)

In [10]:
if(use_cuda):
    model.cuda()

In [11]:
loss_function = nn.NLLLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

In [12]:
load_params(model, './data/params--1.dat')

In [13]:
%%time
epochs = 3

print('starting training')
X_sample = X#[2000:4000]
Y_train_sample = Y_train#[2000:4000]
for i in range(epochs):
    avg_loss = 0.0
    for idx, (x, y_train) in enumerate(zip(X_sample, Y_train_sample)):
        if not idx == 0:
            input_data = get_indexes(x)
            input_data = Variable(LongTensor(input_data))
            y_train = y_train.item()
            target_data = Variable(LongTensor([y_train]))
            hidden = model.init_hidden()
            model.zero_grad()
            y_pred,_ = model(input_data,hidden)
            loss = loss_function(y_pred,target_data)
            avg_loss += loss.data[0]
            
            if idx%500 == 0 or idx == 1:
                print('epoch :%d iterations :%d loss :%g'%(i,idx,loss.data[0]))
            loss.backward()
            optimizer.step()
    save_params(model, i)
    print('the average loss after completion of %d epochs is %g'%((i+1),(avg_loss/len(X_sample))))    

starting training


/home/computer/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
/home/computer/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch :0 iterations :1 loss :0.0430298
epoch :0 iterations :500 loss :0.205671
epoch :0 iterations :1000 loss :0.00860977
epoch :0 iterations :1500 loss :0.0210786
epoch :0 iterations :2000 loss :0.000677109
epoch :0 iterations :2500 loss :0.0103216
epoch :0 iterations :3000 loss :0.000962257
epoch :0 iterations :3500 loss :0.00868368
epoch :0 iterations :4000 loss :0.00387669
epoch :0 iterations :4500 loss :0.00805092
epoch :0 iterations :5000 loss :0.143642
epoch :0 iterations :5500 loss :0.0169215
epoch :0 iterations :6000 loss :0.00094223
epoch :0 iterations :6500 loss :0.000814438
epoch :0 iterations :7000 loss :0.00501537
epoch :0 iterations :7500 loss :0.0525031
epoch :0 iterations :8000 loss :0.00546122
epoch :0 iterations :8500 loss :0.893671
epoch :0 iterations :9000 loss :0.0123391
epoch :0 iterations :9500 loss :0.00272942
the average loss after completion of 1 epochs is 0.292063
epoch :1 iterations :1 loss :0.0676866
epoch :1 iterations :500 loss :0.0246425
epoch :1 iterat

In [15]:
save_params(model, -1)

In [14]:
# torch.save(model.state_dict(), './data/model_last.pth')
# torch.save(model, './data/model_last.pth')

In [15]:
# load_params(model, './data/params.dat')

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.1471  0.0785  0.0784  0.3393  0.5524  0.6816 -0.4937 -0.4309  0.1679

Columns 9 to 17 
   0.2782  0.3115 -0.1261 -0.0107 -1.1822 -0.2314  1.8432 -0.6515 -1.2931

Columns 18 to 26 
   0.2327 -0.0494  2.2513 -0.6735 -1.1014 -1.9060  0.8493 -1.3762  2.7118

Columns 27 to 35 
  -0.7583  0.9625  2.0209  0.0363 -0.3103 -0.1589 -0.7603 -0.0682  0.1950

Columns 36 to 44 
   0.1932 -2.1187 -0.7403  0.0552 -1.2713 -0.5040  0.0629 -1.9660  0.3906

Columns 45 to 53 
  -1.0398 -0.3503  0.0669  1.0941 -1.4916 -1.3252  0.4721  0.0430 -1.8012

Columns 54 to 62 
   0.6825 -0.1782  0.3976  0.4856  0.5159 -0.1171  0.6513  1.1366  0.3853

Columns 63 to 71 
   0.4935 -0.3607  0.9227 -0.8155  1.4683  0.9312  0.6763 -0.7830 -0.4143

Columns 72 to 79 
  -0.5284  0.5089  0.5376  0.3342  0.8130  0.3752  0.0829  0.8094
[torch.cuda.FloatTensor of size 1x1x80 (GPU 0)]

In [14]:
gru = nn.GRU(80,300,dropout=0.2)

In [15]:
gru.weight_hh_l0

Parameter containing:
 5.6124e-03  3.8455e-02  2.5160e-02  ...   5.7284e-02 -4.3757e-02  1.4991e-02
 4.4711e-02  3.2893e-02  4.0117e-02  ...  -6.1036e-03 -5.5793e-02  4.8791e-03
 2.0104e-02  3.2321e-02 -5.7551e-02  ...   2.3971e-03 -2.3227e-02 -3.9308e-02
                ...                   ⋱                   ...                
-4.9176e-02  2.8848e-02 -5.1576e-02  ...  -3.8154e-02  1.0857e-02 -3.1291e-02
 4.2728e-02 -5.5565e-02 -6.8024e-03  ...  -4.2155e-02  4.5272e-02  1.1601e-03
 1.2782e-02 -3.6544e-02 -1.4150e-02  ...   1.3261e-03 -5.3807e-02 -5.2056e-02
[torch.FloatTensor of size 900x300]